In [2]:
import pandas as pd
import os
import collections
import scipy.stats
import numpy as np
import scipy
import csv
import tqdm
import math
from matplotlib import pyplot as plt
import subprocess
%matplotlib inline
import re
from functools import reduce

In [234]:
string_rna="senl32_seq/"
string_mirna="senl32_miRNA/"

In [235]:
path="GMQL_data/"+string_rna
names=[]
barcodes=[]
filelist=os.listdir(path)
filelist.sort()
for file in filelist:

    if file.endswith(".meta"):
        tmp=pd.read_csv(path+file, sep='\t', header=None, index_col=0)
        tmp=tmp.T
        if tmp['biospecimen__shared__bcr_patient_barcode'].values[0] in barcodes:
            print(file)
        else:
            names.append(file[:11])
            barcodes.append(tmp['biospecimen__shared__bcr_patient_barcode'].values[0])

In [236]:
len(barcodes)

33

In [239]:
barcodes=list(barcodes)

In [240]:
len(barcodes)

33

In [241]:
path="GMQL_data/"+string_mirna
ids=[]

filelist=os.listdir(path)
filelist.sort()
for file in filelist:

    if file.endswith(".meta"):
        tmp=pd.read_csv(path+file, sep='\t', header=None, index_col=0)
        tmp=tmp.T
        barcode=tmp['biospecimen__shared__bcr_patient_barcode'].values[0]
        ids.append(barcode)
        #idx=barcodes.index(barcode)
        #order[idx]=file[:11]
        #notorder.append(file[:11])



In [242]:
set1=set(barcodes)
set2=set(ids)
inters=set1.intersection(set2)

barcodes=list(barcodes)
for b in barcodes:
    if b not in inters:
        print(b+' not in index')
        idx=barcodes.index(b)
        n=names[idx]
        names.remove(n)
        barcodes.remove(b)

In [243]:
len(barcodes)

33

In [244]:
len(names)

33

In [245]:
len(inters)

33

In [246]:
path="GMQL_data/"+string_mirna
barcodes=list(barcodes)
notorder=[]
order=np.empty(len(barcodes), dtype="<U30")
filelist=os.listdir(path)
filelist.sort()
for file in filelist:

    if file.endswith(".meta"):
        #print(file)
        tmp=pd.read_csv(path+file, sep='\t', header=None, index_col=0)
        tmp=tmp.T
        barcode=tmp['biospecimen__shared__bcr_patient_barcode'].values[0]
        if barcode in barcodes:
            #print(barcode)
            idx=barcodes.index(barcode)
            order[idx]=file[:11]
            #print(idx)
            #print(file[:11])
            notorder.append(file[:11])


In [250]:
path="GMQL_data/"+string_mirna

col=['patient', 'chrom', 'start', 'stop', 'mirna_id', 'rpm', 'entrez_id', 'gene_symbol']
r=[]
for i in range(len(order)):
    tmp=pd.read_csv(path+order[i], sep='\t', header=None)
    tmp=tmp.values
    for j in range(tmp.shape[0]):
        p='SL_00'
        if i < 10:
            p=p+'00'+str(i)
        elif i< 100:
            p=p+'0'+str(i)
        else:
            p=p+str(i)
        r.append([p, tmp[j][0], tmp[j][1], tmp[j][2], tmp[j][4], tmp[j][6], tmp[j][8], tmp[j][9]])
data=pd.DataFrame(r)
#data.to_csv(path_or_buf ='new_data/sens_mirna.csv', sep='\t', header=col, index=False)
data.head() #ci sono dei nan

,0,1,2,3,4,5,6,7
0,SL_00000,chr1,17368,17436,hsa-mir-6859-1,0.00000,102466751.0,mir6859-1
1,SL_00000,chr1,30365,30503,hsa-mir-1302-2,0.00000,100302278.0,mir1302-2
2,SL_00000,chr1,187890,187958,hsa-mir-6859-2,0.00000,102465909.0,mir6859-2
3,SL_00000,chr1,632324,632413,hsa-mir-6723,0.00000,102465432.0,mir6723
4,SL_00000,chr1,1167103,1167198,hsa-mir-200b,4098.11386,406984.0,mir200b


In [252]:
data.to_csv(path_or_buf ='new_data/senl32_mirna.csv', sep='\t', header=col, index=False)

In [255]:
path="GMQL_data/"+string_rna

col=['patient', 'chrom', 'start', 'stop', 'ensemble_id', 'entrez_id', 'gene_symbol', 'fpkm_uq', 'fpkm']
r=[]
for i in range(len(names)):
    tmp=pd.read_csv(path+names[i], sep='\t', header=None)
    tmp=tmp.values
    for j in range(tmp.shape[0]):
        p='SL_00'
        if i < 10:
            p=p+'00'+str(i)
        elif i< 100:
            p=p+'0'+str(i)
        else:
            p=p+str(i)
        r.append([p, tmp[j][0], tmp[j][1], tmp[j][2], tmp[j][4], tmp[j][5], tmp[j][6], tmp[j][9], tmp[j][10]])
data=pd.DataFrame(r)
#data.to_csv(path_or_buf ='new_data/sens_rna_seq.csv', sep='\t', header=col, index=False)
data.head()

,0,1,2,3,4,5,6,7,8
0,SL_00000,chr1,11868,14409,ENSG00000223972.5,100287102.0,DDX11L1,615.121533,0.027216
1,SL_00000,chr1,14403,29570,ENSG00000227232.5,653635.0,WASH7P,41077.916117,1.817473
2,SL_00000,chr1,17368,17436,ENSG00000278267.1,102465910.0,MIR6859-3,117709.837403,5.208017
3,SL_00000,chr1,29553,31109,ENSG00000243485.3,NaN,RP11-34P13.3,0.000000,0.000000
4,SL_00000,chr1,30365,30503,ENSG00000274890.1,100422831.0,MIR1302-9,0.000000,0.000000


In [256]:
data.to_csv(path_or_buf ='new_data/senl32_rna_seq.csv', sep='\t', header=col, index=False)

In [257]:
res=pd.read_csv('new_data/senl32_mirna.csv', sep='\t')
res.head()

,patient,chrom,start,stop,mirna_id,rpm,entrez_id,gene_symbol
0,SL_00000,chr1,17368,17436,hsa-mir-6859-1,0.00000,102466751.0,mir6859-1
1,SL_00000,chr1,30365,30503,hsa-mir-1302-2,0.00000,100302278.0,mir1302-2
2,SL_00000,chr1,187890,187958,hsa-mir-6859-2,0.00000,102465909.0,mir6859-2
3,SL_00000,chr1,632324,632413,hsa-mir-6723,0.00000,102465432.0,mir6723
4,SL_00000,chr1,1167103,1167198,hsa-mir-200b,4098.11386,406984.0,mir200b


In [258]:
col=['patient', 'chrom', 'start', 'stop', 'mirna_id', 'rpm', 'entrez_id', 'gene_symbol', 'fpkm']

In [165]:
res.to_csv(path_or_buf ='new_data/senl_mirna.csv', sep='\t', header=col, index=False)

In [259]:
fpkm=np.empty(res.shape[0], dtype=float)
for i in range(res.shape[0]):
    rpm=res.iloc[i]['rpm']

    start=res.iloc[i]['start']
    stop=res.iloc[i]['stop']
    fpkm[i]=rpm*100/(2*(stop-start))



In [260]:
res['fpkm']=fpkm
res.head()

,patient,chrom,start,stop,mirna_id,rpm,entrez_id,gene_symbol,fpkm
0,SL_00000,chr1,17368,17436,hsa-mir-6859-1,0.00000,102466751.0,mir6859-1,0.000000
1,SL_00000,chr1,30365,30503,hsa-mir-1302-2,0.00000,100302278.0,mir1302-2,0.000000
2,SL_00000,chr1,187890,187958,hsa-mir-6859-2,0.00000,102465909.0,mir6859-2,0.000000
3,SL_00000,chr1,632324,632413,hsa-mir-6723,0.00000,102465432.0,mir6723,0.000000
4,SL_00000,chr1,1167103,1167198,hsa-mir-200b,4098.11386,406984.0,mir200b,2156.902032


In [261]:
res.to_csv(path_or_buf ='new_data/senl32_mirna.csv', sep='\t', header=col, index=False)

In [140]:
rpm=res.iloc[4]['rpm']
start=res.iloc[4]['start']
stop=res.iloc[4]['stop']
rpm*100/(2*(stop-start))

531.7566921052631

In [ ]:
ciao=pd.read_csv('GMQL_data/sens_miRNA')

In [ ]:
#Create matrices of lenghts of mirna genes
miRNANormallength=MNoKirc.to_matrix(index_regs=['mirna_id', 'gene_symbol'], values_regs=['start','stop'])
miRNACancerlength=MiRNAKirc.to_matrix(index_regs=['mirna_id', 'gene_symbol'], values_regs=['start','stop'])

In [ ]:
#Normalize the mirna data creating fpkm also per mirna from rpm
for i in range (0, 1744):
    miRNANormal.get_values()[i]=(miRNANormal.get_values()[i](10*3))/(2*abs(miRNANormallength.get_values()[i,1]-miRNANormallength.get_values()[i,0]))
    miRNACancer.get_values()[i]=(miRNACancer.get_values()[i](10*3))/(2*abs(miRNACancerlength.get_values()[i,1]-miRNACancerlength.get_values()[i,0]))

In [6]:
path="GMQL_data/res_files/"
names=[]
barcodes=[]
days=[]
rec=[]
filelist=os.listdir(path)
filelist.sort()
for file in filelist:

    if file.endswith(".meta"):
        tmp=pd.read_csv(path+file, sep='\t', header=None, index_col=0)
        tmp=tmp.T
        if tmp['biospecimen__shared__bcr_patient_barcode'].values[0] in barcodes:
            print(file)
        else:
            names.append(file[:11])
            barcodes.append(tmp['biospecimen__shared__bcr_patient_barcode'].values[0])
            if 'clinical__nte__days_to_new_tumor_event_after_initial_treatment' not in tmp.columns:
                days.append('nan')
            else:
                days.append(tmp['clinical__nte__days_to_new_tumor_event_after_initial_treatment'].values[0])
            if 'clinical__clin_shared__regimen_indication' not in tmp.columns:
                rec.append('nan')
            else:
                rec.append(tmp['clinical__clin_shared__regimen_indication'].values[0])

S_00036.gdm.meta
S_00043.gdm.meta
S_00050.gdm.meta
S_00059.gdm.meta


In [ ]:
#non va bene il metadato, devo usare l'excel